<a href="https://colab.research.google.com/github/Dmitri9149/TensorFlow-PyTorch-basics/blob/master/PyTorch_Blocks_and_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import torch
from torch import nn
from torch.nn import functional as F

Some part of the code is based on d2l.ai book: http://d2l.ai/

In [139]:
### basing on the d2l.ai book 

In [140]:
net = nn.Sequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))

X=torch.rand(2,20)
net(X)

tensor([[ 0.2053, -0.0780,  0.2956,  0.0752, -0.1628,  0.1194,  0.1436,  0.1265,
          0.2637, -0.0179],
        [ 0.1205, -0.0068,  0.1768,  0.0200, -0.0235,  0.1234,  0.1002,  0.0918,
          0.1005, -0.0028]], grad_fn=<AddmmBackward>)

In [141]:
net.forward(X)

tensor([[ 0.2053, -0.0780,  0.2956,  0.0752, -0.1628,  0.1194,  0.1436,  0.1265,
          0.2637, -0.0179],
        [ 0.1205, -0.0068,  0.1768,  0.0200, -0.0235,  0.1234,  0.1002,  0.0918,
          0.1005, -0.0028]], grad_fn=<AddmmBackward>)

In [142]:
class MLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # Hidden layer
        self.out = nn.Linear(256, 10)  # Output layer

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))


In [143]:
net = MLP()
net(X)

tensor([[ 0.0605, -0.2953,  0.1275, -0.0947, -0.1352, -0.0898,  0.0836, -0.0462,
         -0.1664,  0.0900],
        [ 0.0755, -0.0501, -0.0735,  0.0169, -0.0793,  0.0262,  0.0168, -0.0598,
         -0.0496,  0.0173]], grad_fn=<AddmmBackward>)

In [144]:
### Custom Sequential Block

In [145]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # Here, `block` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X = block(X)
        return X



In [146]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0691,  0.0445,  0.0539, -0.1580,  0.0480,  0.0225,  0.0473, -0.0732,
         -0.0636,  0.1307],
        [ 0.0958,  0.0267,  0.0650, -0.0870,  0.0343,  0.0545, -0.0105, -0.0932,
         -0.0481,  0.2366]], grad_fn=<AddmmBackward>)

MyParallel model: several models use the same input, executed in paralle and the results of the models are concatenated in one output.

In [147]:
class MyParallel(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # Here, `block` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        list_res=[]
        for block in self._modules.values():
          list_res.append(block(X))
        concat_fin=torch.cat(list_res, dim = -1)
        return concat_fin



In [148]:
list_models = [nn.Linear(20,20),nn.Linear(20,20),nn.Linear(20,20)]

In [149]:
net = MyParallel(nn.Linear(20,20),nn.Linear(20,20),nn.Linear(20,20))
#X = torch.tensor([1,10])
net(X)

tensor([[-1.1848e-01,  9.6248e-02, -7.2014e-02, -1.4919e-01,  2.7896e-01,
         -1.0053e-01, -7.0866e-02, -2.2127e-01, -4.5754e-02, -5.2559e-01,
         -2.8104e-02,  9.4862e-02,  1.9360e-01, -6.2016e-01, -1.7451e-01,
          2.0395e-01,  3.5301e-01, -4.0789e-01,  6.2535e-01,  5.9400e-01,
          3.8014e-01,  6.0317e-02,  3.3547e-01, -1.0027e-03, -7.7176e-01,
         -2.8620e-02,  1.1855e-01, -5.9530e-01, -3.9230e-01,  2.8650e-02,
          1.2174e-01, -3.0434e-01, -6.4093e-01, -2.1674e-01,  4.0402e-01,
          1.1588e-01,  1.2176e-01,  1.0792e-01,  3.8749e-01, -8.1887e-02,
         -5.4010e-01,  1.3976e-01, -1.5026e-01, -5.3061e-03,  5.9215e-01,
          1.3909e-02, -4.3555e-02, -2.7411e-01, -2.6447e-01, -4.3167e-01,
         -2.8847e-01, -1.0698e-01,  1.5624e-01, -9.1585e-02,  3.0593e-01,
         -6.2571e-01, -5.9629e-01, -1.2142e-01, -5.3871e-04, -6.4772e-02],
        [ 2.2551e-03,  1.5969e-01, -2.2191e-01, -2.5465e-01,  4.1350e-01,
         -5.1248e-01, -8.8826e-02,  1

In [150]:
net = MyParallel(*list_models)
net(X)

tensor([[-0.2311, -0.0309, -0.7510,  0.1089,  0.0248,  0.2849, -0.8855, -0.3234,
          0.5289,  0.2630,  0.1437, -0.3050, -0.1439, -0.0525, -0.0218, -0.0265,
         -0.4360,  0.0345, -0.6121,  0.1411, -0.2548,  0.5737, -0.0676, -0.0858,
          0.1464,  0.0964, -0.3200, -0.1280,  0.1782,  0.3212, -0.1296, -0.3389,
         -0.2955, -0.4324, -0.1240,  0.4445,  0.5183, -0.5171, -0.2992,  0.4316,
         -0.2818,  0.4209,  0.0264,  0.1579,  0.0814, -0.2396,  0.5301, -0.3905,
         -0.8701,  0.0459,  0.4915,  0.4948,  0.2061,  0.2438, -1.2019, -0.2246,
          0.2842, -0.5115,  0.1536,  0.9394],
        [-0.2008,  0.1675, -0.2907, -0.2443,  0.0224,  0.4226, -0.6740, -0.0297,
          0.5525, -0.0342,  0.0708,  0.1115, -0.1151,  0.0762,  0.2159, -0.2396,
         -0.1335, -0.0377, -0.4472,  0.1241, -0.0731,  0.3364, -0.2730, -0.3445,
          0.2489,  0.3478,  0.0337, -0.0782, -0.0653,  0.2543, -0.0677, -0.4176,
         -0.2699, -0.4913, -0.0248,  0.3552,  0.3232, -0.2185, 

In [151]:
##### Nesting Modules
class NestedBlocks(nn.Module):
  def __init__(self,*args):
    super().__init__()
    self.net = nn.Sequential(nn.Linear(20,20),nn.Linear(20,20))
    self.linear = nn.Linear(20,20)

  def forward(self, X):
    return self.linear(self.net(X))



In [152]:
tensor = torch.ones([2,20])
net = NestedBlocks()
net(X)

tensor([[ 0.0022, -0.2893,  0.1272, -0.1774,  0.0964,  0.1834,  0.2761, -0.0332,
         -0.1781, -0.0951, -0.0307,  0.0023,  0.2043,  0.1889, -0.1677,  0.1902,
          0.0659, -0.2028,  0.0066,  0.0882],
        [-0.0857, -0.2546,  0.0276, -0.1063,  0.0867,  0.1727,  0.2861, -0.1362,
         -0.1554, -0.0907, -0.0385, -0.0193,  0.2191,  0.2416, -0.2133,  0.2442,
          0.0915, -0.2315, -0.0415,  0.0713]], grad_fn=<AddmmBackward>)

Parameters access

In [153]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X=torch.rand((2,4))
net(X)

tensor([[0.0255],
        [0.0777]], grad_fn=<AddmmBackward>)

In [154]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.3165,  0.0533,  0.1218,  0.2320, -0.2555,  0.1020,  0.0360, -0.2844]])), ('bias', tensor([0.0698]))])


In [155]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0698], requires_grad=True)
tensor([0.0698])


In [156]:
net[2].weight.grad == True

False

In [157]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [158]:
net.state_dict()['0.weight'].data

tensor([[-0.2776,  0.4534, -0.1428, -0.4177],
        [-0.2472, -0.4615,  0.0412,  0.2970],
        [-0.4229, -0.0335,  0.2253, -0.3036],
        [-0.0279,  0.2890, -0.1575, -0.2650],
        [-0.4184, -0.2219,  0.2052, -0.1971],
        [-0.1780, -0.1081,  0.0046, -0.3658],
        [-0.3883,  0.0334,  0.1440,  0.0272],
        [ 0.1154,  0.4347,  0.0013, -0.1815]])

In [159]:
def block1():
  return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                        nn.Linear(8, 4), nn.ReLU())

def block2():
  net = nn.Sequential()
  for i in range(4):
    net.add_module(f'block {i}', block1())
  return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
rgnet(X)

tensor([[0.1538],
        [0.1538]], grad_fn=<AddmmBackward>)

In [160]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [161]:
"""
net_prl = MyParallel(nn.Linear(4,4),nn.Linear(4,4),nn.Linear(4,4))
X=torch.rand(1,4)
net_prl(X)
print(net_prl)
"""

'\nnet_prl = MyParallel(nn.Linear(4,4),nn.Linear(4,4),nn.Linear(4,4))\nX=torch.rand(1,4)\nnet_prl(X)\nprint(net_prl)\n'

In [162]:
rgnet[0][1][0].bias.data

tensor([ 0.2787,  0.2905,  0.1926, -0.4774, -0.4306, -0.3149, -0.4255,  0.3073])